
# Tutorial for conformalized quantile regression (CQR)

We will use the sklearn california housing dataset as the base for the
comparison of the different methods available on MAPIE. Two classes will
be used: :class:`~mapie.quantile_regression.MapieQuantileRegressor` for CQR
and :class:`~mapie.regression.MapieRegressor` for the other methods.

For this example, the estimator will be :class:`~lightgbm.LGBMRegressor` with
``objective="quantile"`` as this is a necessary component for CQR, the
regression needs to be from a quantile regressor.

For the conformalized quantile regression (CQR), we will use a split-conformal
method meaning that we will split the training set into a training and
calibration set. This means using
:class:`~mapie.quantile_regression.MapieQuantileRegressor` with ``cv="split"``
and the ``alpha`` parameter already defined. Recall that the ``alpha`` is
`1 - target coverage`.

For the other type of conformal methods, they are chosen with the
parameter ``method`` of :class:`~mapie.regression.MapieRegressor` and the
parameter ``cv`` is the strategy for cross-validation. In this method, to use a
"leave-one-out" strategy, one would have to use ``cv=-1`` where a positive
value would indicate the number of folds for a cross-validation strategy.
Note that for the jackknife+ after boostrap, we need to use the
class :class:`~mapie.subsample.Subsample` (note that the `alpha` parameter is
defined in the ``predict`` for these methods).


In [1]:
import os
import warnings
from tqdm import tqdm

from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from scipy.stats import randint, uniform
import seaborn as sns
from sklearn.decomposition import PCA

from mapie.regression import MapieRegressor
from mapie.subsample import Subsample
from mapie.quantile_regression import MapieQuantileRegressor

from utils_mapie import get_data, plot_prediction_intervals, get_coverages_widths_by_bins, running_strategies, optimize_estimator

random_state = 7
rng = np.random.default_rng(random_state)

warnings.filterwarnings("ignore")

In [2]:
def running_dataset(name, random_state, alpha=0.2, bins = 10, bins_strategy = "quantile", perc_obs_plot=0.02):
    X, y = get_data(name, random_state=random_state)

    dir_ = "latex_tables/"+name+"/"
    if not os.path.exists(dir_):
        os.mkdir(dir_)

    dir_ = "images/"+name+"/"
    if not os.path.exists(dir_):
        os.mkdir(dir_)

    y_ = pd.DataFrame(y).rename(columns={0: "y"})
    df = pd.concat([X, y_], axis=1)
    pear_corr = df.corr(method='pearson')
    pear_corr.style.background_gradient(cmap='Greens', axis=0)
    pear_corr.to_latex("latex_tables/"+name+"/"+name+"_corr")

    fig, axs = plt.subplots(1, 1, figsize=(5, 5))
    axs.hist(y, bins=50)
    name_y = y_.columns[0]
    axs.set_xlabel(name_y)
    axs.set_title("Histogram of "+name_y)
    plt.savefig("images/"+name+"/"+name+"_histy.png", bbox_inches="tight")
    plt.close()

    pca = PCA(n_components=1)
    pca.fit(X)
    X_ = pca.fit_transform(X)
    df_ = pd.concat([pd.DataFrame(X_).rename(columns={0: "x"}), pd.DataFrame(y).rename(columns={0: "y"})], axis=1)
    sns.histplot(df_, x="x", y=name_y, bins=50)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel("X")
    plt.ylabel("y")
    plt.savefig("images/"+name+"/"+name+"_scatterxy.png", bbox_inches="tight")
    plt.close()

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        random_state=random_state
    )
    X_train, X_calib, y_train, y_calib = train_test_split(
        X_train,
        y_train,
        random_state=random_state
    )

    estimator = LGBMRegressor(
        objective='quantile',
        alpha=0.5,
        random_state=random_state
    )
    params_distributions = dict(
        num_leaves=randint(low=10, high=50),
        max_depth=randint(low=3, high=20),
        n_estimators=randint(low=50, high=300),
        learning_rate=uniform()
    )

    estimator = optimize_estimator(estimator, params_distributions, X_train, y_train)

    STRATEGIES = {
        "naive": {"method": "naive"},
        "cv_plus": {"method": "plus", "cv": 10},
        "jackknife_plus_ab": {"method": "plus", "cv": Subsample(n_resamplings=50)},
        "cqr": {"method": "quantile", "cv": "split", "alpha": alpha},
    }
    y_pred, y_pis, y_test_sorted, y_pred_sorted, lower_bound, upper_bound, coverage, width = running_strategies(STRATEGIES, estimator, X_train, y_train, X_calib, y_calib, X_test, y_test, alpha)

    num_plots = rng.choice(
        len(y_test), int(perc_obs_plot*len(y_test)), replace=False
        )
    fig, axs = plt.subplots(2, 2, figsize=(15, 13))
    coords = [axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]]
    for strategy, coord in zip(STRATEGIES.keys(), coords):
        plot_prediction_intervals(
            strategy,
            coord,
            y_test_sorted[strategy],
            y_pred_sorted[strategy],
            lower_bound[strategy],
            upper_bound[strategy],
            coverage[strategy],
            width[strategy],
            num_plots
            )
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(_, []) for _ in zip(*lines_labels)]
    plt.legend(
        lines[:4], labels[:4],
        # loc='upper center',
        # bbox_to_anchor=(0, -0.15),
        bbox_to_anchor=(0.3, 1.15),
        fancybox=True,
        shadow=True,
        ncol=2
    )
    plt.savefig("images/"+name+"/"+name+"_plotintervals.png", bbox_inches="tight")
    plt.close()

    
    binned_data = get_coverages_widths_by_bins(
        "coverage",
        y_test_sorted,
        y_pred_sorted,
        lower_bound,
        upper_bound,
        STRATEGIES,
        bins=bins,
        bins_strategy=bins_strategy
    )

    values_coverage = binned_data.T.describe()
    mean_coverage = np.array(values_coverage[1:2])
    median_coverage = np.array(values_coverage[5:6])
    std_coverage = np.array(values_coverage[2:3])
    binned_data.T.describe().to_latex("latex_tables/"+name+"/"+name+"_coverage")

    binned_data.T.plot.bar(figsize=(12, 4))
    plt.axhline(0.80, ls="--", color="k")
    plt.ylabel("Conditional coverage")
    plt.xlabel("Binned y values")
    plt.xticks(rotation=345)
    plt.ylim(0.3, 1.0)
    plt.legend(loc=[1, 0])
    plt.savefig("images/"+name+"/"+name+"_barcoverage.png", bbox_inches="tight")
    plt.close()


    binned_data = get_coverages_widths_by_bins(
        "width",
        y_test_sorted,
        y_pred_sorted,
        lower_bound,
        upper_bound,
        STRATEGIES,
        bins=bins,
        bins_strategy=bins_strategy
    )

    values_width = binned_data.T.describe()
    mean_width = np.array(values_width[1:2])
    median_width = np.array(values_width[5:6])
    std_width = np.array(values_width[2:3])
    binned_data.T.describe().to_latex("latex_tables/"+name+"/"+name+"_width")

    binned_data.T.plot.bar(figsize=(12, 4))
    plt.ylabel("Interval width")
    plt.xlabel("Binned y values")
    plt.xticks(rotation=350)
    plt.legend(loc=[1, 0])
    plt.savefig("images/"+name+"/"+name+"_barintervals.png", bbox_inches="tight")
    plt.close()

    data_ = pd.DataFrame(np.hstack([np.array([name]*4).reshape(-1, 1), np.array(["Naive", "CV+", "Jackknife+-AB", "CQR"]).reshape(-1, 1), np.vstack([np.array(list(coverage.values())), list(width.values()), mean_coverage, std_coverage, median_coverage, mean_width, std_width, median_width]).T])).rename(columns={0:"Dataset", 1:"Method", 2:"Coverage", 3:"Width", 4:"Avg. coverage per bin (pb)", 5:"Std. coverage pb", 6:"Median coverage pb", 7:"Avg. width pb", 8:"Std. width pb", 9:"Median width pb"})
    return data_

In [3]:
df = pd.DataFrame()
alpha = 0.2
names = ["california", "diabetes", "make_regression", "friedman1", "friedman2", "friedman3", "heteroscedastic", "homoscedastic"]

for name in tqdm(names):
    perc_obs_plot = 0.02
    if name == "diabetes":
        perc_obs_plot = 0.5

    data_ = running_dataset(name, random_state, alpha=alpha, perc_obs_plot=perc_obs_plot)
    df = df.append(data_, ignore_index=True)

df[["Coverage", "Width", "Avg. coverage per bin (pb)", "Std. coverage pb", "Median coverage pb", "Avg. width pb", "Std. width pb", "Median width pb"]] = df[["Coverage", "Width", "Avg. coverage per bin (pb)", "Std. coverage pb", "Median coverage pb", "Avg. width pb", "Std. width pb", "Median width pb"]].astype(float)

100%|██████████| 8/8 [03:42<00:00, 27.77s/it]


In [4]:
df.groupby("Method").mean().round(4)

,Coverage,Width,Avg. coverage per bin (pb),Std. coverage pb,Median coverage pb,Avg. width pb,Std. width pb,Median width pb
Method,,,,,,,,
CQR,0.7968,32.6041,0.7981,0.0866,0.8007,32.7487,5.4731,32.0965
CV+,0.8727,32.4910,0.8722,0.0935,0.9024,32.4886,0.7878,32.6148
Jackknife+-AB,0.8140,27.0506,0.8136,0.1400,0.8627,27.0511,0.1441,27.0674
Naive,0.6704,18.3041,0.6704,0.1483,0.6821,18.3041,0.0000,18.3041


In [5]:
df_synthetic = df[(df["Dataset"]!="diabetes") & (df["Dataset"]!="california")]
df_real = df[(df["Dataset"]!="diabetes") | (df["Dataset"]!="california")]

In [6]:
df_synthetic.groupby("Method").mean().round(4).to_latex("latex_tables/synthetic_overall")

In [7]:
df.to_csv("latex_tables/cqr_to_other.csv")